<a href="https://colab.research.google.com/github/bhanuchaitu/Secure-Auth-System/blob/main/BheL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.8 MB/s eta 0:00:00


In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive



In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import re
import hashlib
from IPython.display import display
import ipywidgets as widgets
import os

# Define the path to save the file in Google Drive
file_path = '/content/drive/My Drive/credentials.txt'

# Function to hash the password
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# Function to validate the password
def validate_password(password):
    return (len(password) >= 8 and re.search(r"\d", password) and re.search(r"\W", password))

# Function to check if a username already exists and return the stored password hash
def get_stored_password(username):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith(f"Username: {username}, Password:"):
                    return line.split("Password:")[1].strip()
    return None

# Function to update password for a specific username
def update_password(username, new_password):
    new_password_hash = hash_password(new_password)
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()
        with open(file_path, 'w') as file:
            for line in lines:
                if line.startswith(f"Username: {username},"):
                    file.write(f"Username: {username}, Password: {new_password_hash}\n")
                else:
                    file.write(line)

# Function to save credentials to Google Drive
def save_credentials(username, password):
    password_hash = hash_password(password)
    with open(file_path, 'a') as file:
        file.write(f"Username: {username}, Password: {password_hash}\n")

# Function to handle the signup button click
def on_signup_button_clicked(b):
    username = signup_username_input.value
    password = signup_password_input.value
    confirm_password = confirm_password_input.value

    if get_stored_password(username):
        output.clear_output()
        with output:
            print("Username already exists. Redirecting to login page...")
        switch_to_login_form()
    elif password != confirm_password:
        output.clear_output()
        with output:
            print("Passwords do not match. Please try again.")
    elif not validate_password(password):
        output.clear_output()
        with output:
            print("Invalid password. It must contain at least 8 characters, including a special character and a number.")
    else:
        save_credentials(username, password)
        output.clear_output()
        with output:
            print(f"Welcome, {username}! Your account has been created.")
        switch_to_login_form()

# Function to handle the login button click on the login page
def on_login_button_clicked(b):
    username = username_input.value
    password = password_input.value

    stored_password_hash = get_stored_password(username)
    if stored_password_hash:
        if hash_password(password) == stored_password_hash:
            output.clear_output()
            with output:
                print(f"Welcome back, {username}!")
        else:
            output.clear_output()
            with output:
                print("Incorrect password. Please try again.")
    else:
        output.clear_output()
        with output:
            print("Username not found. Please check and try again or sign up.")

# Function to handle the "Login" button click on the signup page
def on_signup_login_button_clicked(b):
    switch_to_login_form()

# Function to handle the forgot password button click
def on_forgot_password_button_clicked(b):
    switch_to_forgot_password_form()

# Function to handle the reset password button click
def on_reset_password_button_clicked(b):
    forgot_username = forgot_username_input.value
    new_password = forgot_password_input.value

    if get_stored_password(forgot_username):
        if validate_password(new_password):
            update_password(forgot_username, new_password)
            output.clear_output()
            with output:
                print("Password updated successfully.")
            switch_to_login_form()
        else:
            output.clear_output()
            with output:
                print("Invalid password. It must contain at least 8 characters, including a special character and a number.")
    else:
        output.clear_output()
        with output:
            print("Username not found. Please check and try again.")

# Function to switch to the signup form
def switch_to_signup_form():
    login_form.layout.display = 'none'
    forgot_password_form.layout.display = 'none'
    signup_form.layout.display = 'block'
    output.clear_output()

# Function to switch to the login form
def switch_to_login_form():
    signup_form.layout.display = 'none'
    forgot_password_form.layout.display = 'none'
    login_form.layout.display = 'block'
    output.clear_output()

# Function to switch to the forgot password form
def switch_to_forgot_password_form():
    signup_form.layout.display = 'none'
    login_form.layout.display = 'none'
    forgot_password_form.layout.display = 'block'
    output.clear_output()

# Create input fields for signup
signup_username_input = widgets.Text(placeholder='Enter your username', description='Username:')
signup_password_input = widgets.Password(placeholder='Enter your password', description='Password:')
confirm_password_input = widgets.Password(placeholder='Confirm your password', description='Confirm Password:')
signup_button = widgets.Button(description='Sign Up')
signup_login_button = widgets.Button(description='Already have an account? Login')

# Attach event handler to the signup button
signup_button.on_click(on_signup_button_clicked)
# Attach event handler to the "Already have an account? Login" button
signup_login_button.on_click(on_signup_login_button_clicked)

# Create signup form
signup_form = widgets.VBox([signup_username_input, signup_password_input, confirm_password_input, signup_button, signup_login_button])

# Create input fields for login
username_input = widgets.Text(placeholder='Enter your username', description='Username:')
password_input = widgets.Password(placeholder='Enter your password', description='Password:')
login_button = widgets.Button(description='Login')
forgot_password_button = widgets.Button(description='Forgot Password')

# Attach event handler to the login button
login_button.on_click(on_login_button_clicked)
# Attach event handler to the forgot password button
forgot_password_button.on_click(on_forgot_password_button_clicked)

# Create login form
login_form = widgets.VBox([username_input, password_input, login_button, forgot_password_button])

# Create input fields for forgot password
forgot_username_input = widgets.Text(placeholder='Enter your username', description='Username:')
forgot_password_input = widgets.Password(placeholder='Enter your new password', description='New Password:')
reset_password_button = widgets.Button(description='Reset Password')

# Attach event handler to the reset password button
reset_password_button.on_click(on_reset_password_button_clicked)

# Create forgot password form
forgot_password_form = widgets.VBox([forgot_username_input, forgot_password_input, reset_password_button])

# Create output area
output = widgets.Output()

# Initially, show the signup form and hide others
signup_form.layout.display = 'block'
login_form.layout.display = 'none'
forgot_password_form.layout.display = 'none'

# Display the widgets
display(signup_form, login_form, forgot_password_form, output)

Output()